#Text rank#
To remember
* cite resources appropriately later

In [0]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx 


In [0]:
# download and required modules of nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# setting up gdrive 
from google.colab import drive
drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Dataset : tmp#
Description,size


In [0]:
# testing phase 
# will change later to more complex 
# this cell must be well abstracted


# read the data as csv
# data = pd.read_csv("content/gdrive/My Drive/submit")
data=pd.read_csv("/content/gdrive/My Drive/tennis_articles_v4.csv")

In [0]:
# splitting into sentences
sentences = []
for s in data["article_text"]:
  sentences.extend(sent_tokenize(s))

In [0]:
#print sample sentences
_=[print(s) for s in sentences[:5]]

Maria Sharapova has basically no friends as tennis players on the WTA Tour.
The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.
I think everyone knows this is my job here.
When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
I'm a pretty competitive girl.


#Initialising Word Embeddings(Glove)#

In [0]:
# Uncomment below commands to download zip
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip.1

In [0]:
word_emb = dict()
with open('glove.6B.100d.txt',encoding='utf-8') as f:
  for line in f :
    tokens = line.strip().split()
    word_emb[tokens[0]] = np.asarray(tokens[1:],dtype='float32')
print("Initialised "+str(len(word_emb))+" embeddings")

Initialised 400000 embeddings


#Preprocessing Stage#
* Implement alternate methods for calculating sentence vector.

In [0]:
# text cleaning 
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]"," ")
clean_sentences = [s.lower() for s in clean_sentences]

# remove stop_words
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
stop_words = stopwords.words('english')
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [0]:
# vectorizing sentences
# we take the average of word vectors as sentence vector
# note that word vectors are of length 100
zero_vec = np.zeros((100,))
sentence_vectors = []
for s in clean_sentences:
  if len(s) >0 :
    words = s.split()
    avg_vec = np.mean([word_emb.get(w,zero_vec) for w in words],0)
    sentence_vectors.append(avg_vec)
  else:
    sentence_vectors.append(zero_vec)

# convert to np array
sentence_vectors = np.array(sentence_vectors)

#Similarity Matrix#
* Implement cosine,unit overlap, and others

In [0]:
num_sentences = sentence_vectors.shape[0]
similarity_matrix = np.zeros((num_sentences,)*2)
for i in range(num_sentences):
  for j in range(num_sentences):
    if i==j:
      # ignore self similarity
      continue;
    similarity_matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                                sentence_vectors[j].reshape(1,100))[0][0]

                                                
      


#Applying Ranking Algorithm#


In [0]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

#Results#
* print 
* rouge and other measures to evaluate performance

In [0]:
for i in range(10):
  print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 